In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
import os, cv2, random #,re
from sklearn.model_selection import train_test_split
import tqdm
import torch
import torch.nn as nn
import torch.utils.data as Data
import torchvision      # 数据库模块
import torchvision.transforms as transforms

import matplotlib.pyplot as plt

In [ ]:
train_dir = "../input/train/train/"
test_dir = "../input/test/test/"
train_df = pd.read_csv('../input/train.csv')
print(train_df.head(),'\n')
print(train_df['has_cactus'].value_counts(),'\n')

print('Num train samples:{0}'.format(len(os.listdir(train_dir))))
print('Num test samples:{0}'.format(len(os.listdir(test_dir))))

In [ ]:
fig,ax = plt.subplots(1,5,figsize=(15,3))
for i, idx in enumerate(train_df[train_df['has_cactus']==1]['id'][-5:]):
    ax[i].imshow(cv2.imread(train_dir+idx)) # [...,[2,1,0]]
    
fig,ax = plt.subplots(1,5,figsize=(15,3))
for i, idx in enumerate(train_df[train_df['has_cactus']==0]['id'][-5:]):
    ax[i].imshow(cv2.imread(train_dir+idx)) # [...,[2,1,0]]

In [ ]:
np.shape(cv2.imread(train_dir+idx))

In [ ]:
# Libreries

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torchvision
import torchvision.transforms as transforms

from sklearn.model_selection import train_test_split

In [ ]:
# I Should learn to use this part in the future!!!

# NOTE: class is inherited from Dataset !!!!!!!
class MyDataset(Dataset):
    def __init__(self, df_data, data_dir = './', transform=None):
        super().__init__() #The super function in Python can be used to gain access to inherited methods which is either from a parent (Dataset) or sibling class. 
        self.df = df_data.values
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        img_name,label = self.df[index]
#         img_path = os.path.join(self.data_dir, img_name)
#         image = cv2.imread(img_path)
        image = cv2.imread(self.data_dir+img_name)
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
# Hyper parameters
num_epochs = 25
num_classes = 2
batch_size = 128
learning_rate = 0.002

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
train, val = train_test_split(train_df, stratify=train_df.has_cactus, test_size=0.1,random_state = 42) # stratify=train_df.has_cactus: 依照train_df.has_cactus中0，1比例来保持train&val中的0，1比例
print(train.shape, val.shape)

# transforms see: https://pytorch.org/docs/stable/torchvision/transforms.html
trans_train = transforms.Compose([transforms.ToPILImage(), #Converts a torch.*Tensor of shape C x H x W or a numpy ndarray 
                                                           #of shape H x W x C to a PIL Image while preserving the value range.
                                  transforms.Pad(32, padding_mode='reflect'), # figure size becomes (32*3,32*3)
                                  transforms.Resize(224), # resize to (224*224)
                                  transforms.ToTensor(),#Converts a PIL Image or numpy.ndarray (H x W x C) in the range [0, 255] 
                                                        #to a torch.FloatTensor of shape (C x H x W) in the range [0.0, 1.0]
                                  transforms.Normalize(mean=[0.5, 0.5, 0.5],std=[0.5, 0.5, 0.5])])

dataset_train = MyDataset(df_data=train, data_dir=train_dir, transform=trans_train)
dataset_valid = MyDataset(df_data=val, data_dir=train_dir, transform=trans_train)

loader_train = DataLoader(dataset = dataset_train, batch_size=batch_size, shuffle=True, num_workers=0)
loader_valid = DataLoader(dataset = dataset_valid, batch_size=batch_size//2, shuffle=False, num_workers=0)

In [ ]:
# The Model is too big to run on Kaggle server....!!!!!!!!!!!

# VGG 16
# NOTE: class is inherited from nn.Module
class SimpleCNN(nn.Module):
    def __init__(self):
        # ancestor constructor call
        super(SimpleCNN, self).__init__()
        
        self.conv1 = nn.Sequential(  # input shape (3, 96, 96)
            nn.Conv2d(
                in_channels=3,      # input height
                out_channels=64,    # n_filters
                kernel_size=3,      # filter size
                stride=1,           # filter movement/step
                padding=1,      # padding size
            ),      # output shape (64, 224,224)
            nn.BatchNorm2d(224), #batch normalization
            nn.ReLU(),    # activation
            nn.Conv2d(
                in_channels=64,      # input height
                out_channels=64,    # n_filters
                kernel_size=3,      # filter size
                stride=1,           # filter movement/step
                padding=1,      # padding size
            ),      # output shape (64, 224,224)
            nn.BatchNorm2d(224), #batch normalization
            nn.ReLU(),    # activation
            nn.MaxPool2d(kernel_size=2, stride=2),    # 在 2x2 空间里向下采样, output shape (64, 112, 112)
        )
        
        self.conv2 = nn.Sequential(  # input shape (32, 49, 49)
            nn.Conv2d(64,128,3,1,1),      # output shape (128*112*112)
            nn.BatchNorm2d(112), #batch normalization
            nn.ReLU(),    # activation
            nn.Conv2d(128,128,3,1,1),      # output shape (128*112*112)
            nn.BatchNorm2d(112), #batch normalization
            nn.ReLU(),    # activation
            nn.MaxPool2d(2, 2),    # 在 2x2 空间里向下采样, output shape (128, 56, 56)
        )
        
        self.conv3 = nn.Sequential(  # input shape (64,51,51)
            nn.Conv2d(128,256,3,1,1),      # output shape (256,56,56)
            nn.BatchNorm2d(56), #batch normalization
            nn.ReLU(),    # activation
            nn.Conv2d(256,256,3,1,1),      # output shape (256,56,56)
            nn.BatchNorm2d(56), #batch normalization
            nn.ReLU(),    # activation
            nn.MaxPool2d(2, 2),    # 在 2x2 空间里向下采样, output shape (256, 28, 28)
        )
        
        self.conv4 = nn.Sequential(  # input shape (128, 13, 13)
            nn.Conv2d(256,512,3,1,1),      # output shape (512,28,28)
            nn.BatchNorm2d(28), #batch normalization
            nn.ReLU(),    # activation
            nn.Conv2d(512,512,3,1,1),      # output shape (512,28,28)
            nn.BatchNorm2d(28), #batch normalization
            nn.ReLU(),    # activation
            nn.MaxPool2d(2, 2),    # 在 2x2 空间里向下采样, output shape (512, 14, 14)
        )
        
        self.conv5 = nn.Sequential(  # input shape (256, 7, 7)
            nn.Conv2d(512,512,3,1,1),      # output shape (512,14,14)
            nn.BatchNorm2d(14), #batch normalization
            nn.ReLU(),    # activation
            nn.Conv2d(512,512,3,1,1),      # output shape (512,14,14)
            nn.BatchNorm2d(14), #batch normalization
            nn.ReLU(),    # activation
            nn.Conv2d(512,512,3,1,1),      # output shape (512,14,14)
            nn.BatchNorm2d(14), #batch normalization
            nn.ReLU(),    # activation
            nn.MaxPool2d(2, 2),    # 在 2x2 空间里向下采样, output shape (512,7,7)
        )
        
        self.fc1 = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(),    # activation
            nn.Linear(4096, 4096),
            nn.ReLU(),    # activation
            nn.Linear(4096, 1000),
            nn.ReLU()   # activation
        )
                                 
        self.fc2 = nn.Sequential(
            nn.Linear(1000, 2),
            nn.Softmax()
        )
        
    def forward(self, x):
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        #print(x.shape) # lifehack to find out the correct dimension for the Linear Layer
        x = x.view(-1, 512 * 7 * 7) # !!!
        x = self.fc1(x)
        x = self.fc2(x)
        return x
    

In [ ]:
# use pre-trained model instead
from torchvision import models
model = models.vgg16_bn(pretrained = True)


In [ ]:
model = model.to(device)

# Loss and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.0001, amsgrad=True, weight_decay=1e-4)

# Train the model
total_step = len(loader_train)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(loader_train):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))
            correct = 0
            total = 0
            with torch.no_grad():
                for steptemp,(valimages, vallabels) in enumerate(loader_valid):
                    valimages,vallabels = valimages.cuda(),vallabels.cuda()
                    # print(steptemp,'/25')
                    outputs = model(valimages)
                    _, predicted = torch.max(outputs.data, 1)
                    total += vallabels.size(0)
                    correct += (torch.squeeze(predicted.type(torch.LongTensor).cuda()) == vallabels).sum().item()
            print(f'Accuracy of the network on the {total} test images: %.5f %%' % (100 * correct / total))

In [ ]:
# generator for test data 
sub = pd.read_csv('../input/sample_submission.csv')

dataset_valid = MyDataset(df_data=sub, data_dir=test_dir, transform=trans_train)
loader_test = DataLoader(dataset = dataset_valid, batch_size=32, shuffle=False, num_workers=0)
model.eval()

preds = []
for batch_i, (data, target) in enumerate(loader_test):
    data, target = data.cuda(), target.cuda()
    output = model(data)

    pr = output[:,1].detach().cpu().numpy()
    for i in pr:
        preds.append(i)

sub['has_cactus'] = preds
sub.to_csv('submission.csv', index=False)